In [1]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append(r'.\Pycharm')
from Captcha import gen_captcha,numbers

def text2vec(text):#text变为one hot coding
    vector = np.zeros(40)
    for i in range(len(text)):
        idx = i*10+ord(text[i])-48#ACSII转int
        vector[idx] = 1
    return vector

def gen_batch(size=100):#生成一批数据
    batch_x = np.zeros((size,9600))
    batch_y = np.zeros((size,40))
    for i in range(size):
        text,image,data = gen_captcha()
        batch_x[i,:] = np.mean(image,-1).flatten()/255#rgb转为灰度值，再归一化
        batch_y[i,:] = text2vec(text)#text变为one hot coding
    return batch_x,batch_y
    
with tf.name_scope('input') as scope:#name_scope可以让变量有相同的命名，只是限于tf.Variable的变量，可以看做程序里的一个单元
    x = tf.placeholder(tf.float32,[None,60*160],name='x')#N张图片，每张图片像素60*160，放入灰度图
    y_ = tf.placeholder(tf.float32,[None,4*10],name='y_')#one hot coding 6;[0,0,0,0,0,0,1,0,0,0],每个y_ 4个数字，每个数字用1*10向量表示
    
with tf.name_scope('net') as scope:
    with tf.name_scope('layer1_hidden1') as scope:
        w0 = tf.Variable(tf.random_normal([9600,1024]),name = 'w')
        b0 = tf.Variable(tf.random_normal([1024]),name = 'b')
        a0 = tf.matmul(x,w0)+b0
        
    with tf.name_scope('layer1_hidden2') as scope:#可以用for循环增加层数
        w1 = tf.Variable(tf.random_normal([1024,1024]),name = 'w')
        b1 = tf.Variable(tf.random_normal([1024]),name = 'b')
        a1 = tf.matmul(a0,w1)+b1
        
    with tf.name_scope('layer2_output') as scope:
        w = tf.Variable(tf.random_normal([1024,40]),name = 'w')
        b = tf.Variable(tf.random_normal([40]),name = 'b')
        y = tf.matmul(a1,w)+b
    
with tf.name_scope('training') as scope:
    #loss = tf.losses.mean_squared_error(y_, y)
    loss = tf.losses.sigmoid_cross_entropy(y_,y)
    tf.summary.scalar('loss',loss)#记录并统计loss
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
    
with tf.name_scope('predict') as scope:
    p = tf.argmax(tf.reshape(y,[-1,4,10]),2) #在第二个维度将one hot coding转为十进制数，投票的方式
    label = tf.argmax(tf.reshape(y_,[-1,4,10]),2)
    pred = tf.equal(p,label)#判断预测和真实值是否相等
    accuracy = tf.reduce_mean(tf.cast(pred,tf.float32))
    tf.summary.scalar('accuracy',accuracy)
 
       
with tf.Session() as sess:
    merged = tf.summary.merge_all()    
    tf.global_variables_initializer().run()
    writer = tf.summary.FileWriter('./log/demo14',sess.graph)
    
    
    step=0
    for i in range(5000):
        #训练
        x_train,y_train = gen_batch(64)
        _,loss_ = sess.run([optimizer,loss],feed_dict={x:x_train,y_:y_train})
        #print(step,/loss_)
        #每100批测试一次准确率
        if step%50 == 0:
            x_test,y_test = gen_batch(50)
            acc = sess.run([accuracy], feed_dict={x:x_test, y_:y_test})
            #summary,acc = sess.run([merged,accuracy],feed_dict={x:x_test,y_:y_test})
            #writer.add_summary(summary,step)
            print(acc)
        step = step + 1
        
        

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[0.095]
[0.075]
[0.13]
[0.11]
[0.115]
[0.125]
[0.14]
[0.145]
[0.125]
[0.19]
[0.135]
[0.115]
[0.22]
[0.17]
[0.205]
[0.2]
[0.18]
[0.235]
[0.2]
[0.21]
[0.21]
[0.225]
[0.175]
[0.27]
[0.285]
[0.205]
[0.26]
[0.195]
[0.305]
[0.27]
[0.255]
[0.35]
[0.3]
[0.315]
[0.26]
[0.225]
[0.22]
[0.29]
[0.22]
[0.305]
[0.27]
[0.31]
[0.29]
[0.24]


KeyboardInterrupt: 